In [ ]:
# ==============================================================================
#           PROTÓTIPO V2: SISTEMA NEURO-SIMBÓLICO COM META-COGNIÇÃO
#               Implementando Re-estratégia Dinâmica e Regras Flexíveis
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. Imports e Configurações (sem alterações)
# ------------------------------------------------------------------------------
import numpy as np
import re
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
from enum import Enum
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ------------------------------------------------------------------------------
# 2. Estruturas de Dados e Enums (sem alterações)
# ------------------------------------------------------------------------------

class ReasoningStrategy(Enum):
    NEURAL_FAST = "neural_fast"
    SYMBOLIC_CAREFUL = "symbolic_careful"
    HYBRID_BALANCED = "hybrid_balanced"

@dataclass
class MetaCognitionState:
    current_strategy: ReasoningStrategy = ReasoningStrategy.NEURAL_FAST
    confidence_in_decision: float = 0.0
    reasoning_trace: List[str] = field(default_factory=list)
    uncertainty_sources: List[str] = field(default_factory=list)
    context_complexity: float = 0.0
    attempted_strategies: List[ReasoningStrategy] = field(default_factory=list) # ### MELHORIA ###: Rastreia estratégias já tentadas

# ------------------------------------------------------------------------------
# 3. A Classe Principal: O Agente Meta-Cognitivo (com modificações)
# ------------------------------------------------------------------------------

class NeuroSymbolicMetaCognition:
    def __init__(self, neural_confidence_threshold: float = 0.75):
        self.neural_confidence_threshold = neural_confidence_threshold
        self.symbolic_rules = self._initialize_symbolic_rules()
        self.reset_state()

    def reset_state(self):
        self.meta_state = MetaCognitionState()
        logging.info("Estado meta-cognitivo resetado para um novo problema.")

    def _initialize_symbolic_rules(self) -> Dict:
        """ ### MELHORIA 1: Regras Simbólicas Mais Flexíveis ### """
        return {
            "mathematical": {
                # REGRA ANTIGA (muito rígida): r'Se x \+ (\d+) = (\d+), qual é o valor de x\?'
                # REGRA NOVA (flexível): Busca pelo padrão 'x + num = num' em qualquer lugar do texto.
                # \s* permite espaços variáveis. (?i) torna a busca case-insensitive.
                "extract_linear_equation": r'(?i)x\s*\+\s*(\d+)\s*=\s*(\d+)'
            },
            "logical": {
                "check_premise_conclusion_validity": True
            }
        }

    def solve_problem(self, problem: str, context: Dict) -> Dict:
        self.reset_state()
        self._meta_analyze_problem(problem, context)

        # O ciclo de execução agora é dinâmico e pode mudar de estratégia
        solution = self._execute_reasoning_cycle(problem)

        self._meta_evaluate_solution(solution, problem)

        return {"solution": solution, "meta_state": self.meta_state}

    def _meta_analyze_problem(self, problem: str, context: Dict):
        self.meta_state.reasoning_trace.append("META: Iniciando análise do problema.")
        complexity_factors = {
            "is_technical": self._is_problem_technical(problem),
            "has_ambiguity": self._detect_ambiguity(problem)
        }
        self.meta_state.context_complexity = np.mean(list(complexity_factors.values()))
        self.meta_state.reasoning_trace.append(f"META: Complexidade calculada: {self.meta_state.context_complexity:.2f}")
        if self.meta_state.context_complexity > 0.6:
            self.meta_state.reasoning_trace.append("META: Alerta! Detectei alta complexidade. A abordagem padrão pode falhar.")
            self.meta_state.uncertainty_sources.append("high_initial_complexity")

    def _choose_initial_strategy(self, problem: str) -> ReasoningStrategy:
        """Escolhe a *primeira* estratégia a ser tentada."""
        self.meta_state.reasoning_trace.append("META: Decidindo a estratégia de raciocínio *inicial*.")

        # A lógica de escolha inicial permanece, mas agora é apenas um ponto de partida.
        if "matemática" in problem.lower() or "equação" in problem.lower() or self.meta_state.context_complexity > 0.7:
             # Prioriza o simbólico para problemas que parecem técnicos
            strategy = ReasoningStrategy.SYMBOLIC_CAREFUL
        elif self.meta_state.context_complexity < 0.3:
            strategy = ReasoningStrategy.NEURAL_FAST
        else:
            strategy = ReasoningStrategy.HYBRID_BALANCED

        self.meta_state.reasoning_trace.append(f"META: Estratégia inicial escolhida: {strategy.value}")
        return strategy

    def _execute_reasoning_cycle(self, problem: str) -> str:
        """
        ### MELHORIA 2 & 3: Ciclo de Re-estratégia Dinâmica ###
        Executa, avalia e, se necessário, tenta uma nova estratégia (fallback).
        """
        initial_strategy = self._choose_initial_strategy(problem)
        self.meta_state.current_strategy = initial_strategy

        max_retries = 2
        for attempt in range(max_retries):
            # Adiciona a estratégia atual à lista de tentativas
            self.meta_state.attempted_strategies.append(self.meta_state.current_strategy)

            # Executa a estratégia atual
            strategy = self.meta_state.current_strategy
            if strategy == ReasoningStrategy.NEURAL_FAST:
                solution = self._neural_reasoning(problem)
            elif strategy == ReasoningStrategy.SYMBOLIC_CAREFUL:
                solution = self._symbolic_reasoning(problem)
            else: # HYBRID_BALANCED
                solution = self._hybrid_reasoning(problem)

            # Avalia se a estratégia falhou e se uma nova tentativa é necessária
            if self._should_retry(attempt, max_retries):
                # Se a estratégia falhou, o sistema decide qual será a próxima
                new_strategy = self._re_strategize()
                if new_strategy:
                    self.meta_state.current_strategy = new_strategy
                else:
                    # Nenhuma outra estratégia para tentar, encerra o ciclo
                    break
            else:
                # A estratégia foi bem-sucedida ou não há mais tentativas, encerra o ciclo
                return solution

        return solution # Retorna a solução da última tentativa

    def _should_retry(self, attempt, max_retries):
        """META-COGNITIVO: Avalia se o resultado da estratégia atual é inaceitável e se deve tentar de novo."""
        if attempt + 1 >= max_retries:
            return False # Esgotou as tentativas

        # Condições de falha que disparam uma nova tentativa
        if "no_symbolic_rule_matched" in self.meta_state.uncertainty_sources:
            return True
        # Poderíamos adicionar mais condições aqui, ex: confiança muito baixa

        return False

    def _re_strategize(self) -> Optional[ReasoningStrategy]:
        """META-COGNITIVO: Escolhe uma nova estratégia de fallback."""
        self.meta_state.reasoning_trace.append(f"META: A estratégia '{self.meta_state.current_strategy.value}' falhou. Re-avaliando...")

        # Regras de Fallback
        # Se a simbólica falhou, a melhor aposta é a híbrida, que pode usar o neural para entender e o simbólico para validar.
        if self.meta_state.current_strategy == ReasoningStrategy.SYMBOLIC_CAREFUL and ReasoningStrategy.HYBRID_BALANCED not in self.meta_state.attempted_strategies:
            self.meta_state.reasoning_trace.append("META: Fallback -> Tentando a estratégia HÍBRIDA.")
            return ReasoningStrategy.HYBRID_BALANCED

        # Se a neural teve baixa confiança, a híbrida é a melhor para verificação.
        if self.meta_state.current_strategy == ReasoningStrategy.NEURAL_FAST and ReasoningStrategy.HYBRID_BALANCED not in self.meta_state.attempted_strategies:
            self.meta_state.reasoning_trace.append("META: Fallback -> Tentando a estratégia HÍBRIDA para verificação.")
            return ReasoningStrategy.HYBRID_BALANCED

        self.meta_state.reasoning_trace.append("META: Nenhuma estratégia de fallback disponível. Encerrando ciclo de tentativas.")
        return None # Nenhuma nova estratégia para tentar


    def _meta_evaluate_solution(self, solution: str, original_problem: str):
        # A lógica aqui permanece a mesma, mas agora ela opera sobre o resultado final do ciclo.
        self.meta_state.reasoning_trace.append("META: Iniciando avaliação final da solução e do processo.")
        if len(self.meta_state.uncertainty_sources) > 0:
            self.meta_state.reasoning_trace.append(f"META: REFLEXÃO: O processo teve fontes de incerteza ({self.meta_state.uncertainty_sources}). A confiança deve ser ajustada.")
            self.meta_state.confidence_in_decision *= (1.0 - 0.2 * len(self.meta_state.uncertainty_sources))
            self.meta_state.reasoning_trace.append(f"META: Confiança final ajustada para: {self.meta_state.confidence_in_decision:.2f}")
        else:
            self.meta_state.reasoning_trace.append("META: REFLEXÃO: O processo de raciocínio foi limpo, sem fontes de incerteza detectadas.")

    def _neural_reasoning(self, problem: str) -> str:
        self.meta_state.reasoning_trace.append("NEURAL: Iniciando processamento rápido baseado em padrões.")

        # ### MELHORIA ###: Simulação um pouco mais inteligente para extrair o número
        match = re.search(r'(\d+)\s*\.\s*Está certo\?', problem)
        if match:
            # Simula a resposta a uma pergunta de verificação
            solution = f"Sim, a resposta parece ser {match.group(1)}."
            neural_confidence = 0.85
        elif "x +" in problem:
            # Simula uma resposta intuitiva, mas possivelmente incompleta
            nums = re.findall(r'\d+', problem)
            if len(nums) == 2:
                solution = f"A resposta é {int(nums[1]) - int(nums[0])}."
            else:
                solution = "A resposta é 3." # Fallback genérico
            neural_confidence = 0.90
        else:
            solution = f"Resposta intuitiva para '{problem}'."
            neural_confidence = 0.60

        self.meta_state.reasoning_trace.append(f"NEURAL: Resposta gerada com confiança de {neural_confidence:.2f}")
        self.meta_state.confidence_in_decision = neural_confidence
        if neural_confidence < self.neural_confidence_threshold:
            self.meta_state.reasoning_trace.append(f"META: Alerta! Confiança neural ({neural_confidence:.2f}) abaixo do limiar ({self.neural_confidence_threshold}).")
            self.meta_state.uncertainty_sources.append("low_neural_confidence")
        return solution

    def _symbolic_reasoning(self, problem: str) -> str:
        """Usa a nova regra flexível."""
        self.meta_state.reasoning_trace.append("SYMBOLIC: Iniciando raciocínio baseado em regras flexíveis.")

        rule = self.symbolic_rules["mathematical"]["extract_linear_equation"]
        match = re.search(rule, problem)

        if match:
            # Limpa incertezas anteriores se encontrou uma regra
            if "no_symbolic_rule_matched" in self.meta_state.uncertainty_sources:
                self.meta_state.uncertainty_sources.remove("no_symbolic_rule_matched")

            self.meta_state.reasoning_trace.append("SYMBOLIC: Regra 'extract_linear_equation' aplicada com sucesso.")
            a = int(match.group(1))
            b = int(match.group(2))

            step1 = f"Equação extraída: x + {a} = {b}"
            step2 = f"Subtraindo {a} de ambos os lados: x = {b} - {a}"
            result = b - a
            step3 = f"Solução: x = {result}"

            self.meta_state.reasoning_trace.extend([f"SYMBOLIC (Passo 1): {step1}", f"SYMBOLIC (Passo 2): {step2}", f"SYMBOLIC (Passo 3): {step3}"])

            self.meta_state.confidence_in_decision = 0.98
            return f"O valor de x é {result}."
        else:
            self.meta_state.reasoning_trace.append("SYMBOLIC: Nenhuma regra matemática aplicável encontrada no texto.")
            if "no_symbolic_rule_matched" not in self.meta_state.uncertainty_sources:
                 self.meta_state.uncertainty_sources.append("no_symbolic_rule_matched")
            self.meta_state.confidence_in_decision = 0.1
            return "Não foi possível resolver o problema com as regras simbólicas atuais."

    def _hybrid_reasoning(self, problem: str) -> str:
        # A lógica interna do Híbrido pode ser simplificada, pois o ciclo externo agora gerencia os fallbacks.
        # Ele age como um "consultor especialista" que usa ambos os métodos e reporta.
        self.meta_state.reasoning_trace.append("HYBRID: Iniciando coordenação entre módulos neural e simbólico.")

        # Executa ambos os módulos para comparação
        neural_solution = self._neural_reasoning(problem)
        symbolic_solution = self._symbolic_reasoning(problem)

        # Compara os resultados
        num_neural = re.findall(r'\d+', neural_solution)
        num_symbolic = re.findall(r'\d+', symbolic_solution)

        if num_neural and num_symbolic and num_neural[-1] == num_symbolic[-1]:
            self.meta_state.reasoning_trace.append("META: Concordância entre neural e simbólico. Confiança alta.")
            self.meta_state.confidence_in_decision = 0.99
            # Remove incerteza de discordância se ela foi adicionada por engano
            if "neural_symbolic_disagreement" in self.meta_state.uncertainty_sources:
                self.meta_state.uncertainty_sources.remove("neural_symbolic_disagreement")
            return symbolic_solution # Retorna a solução simbólica, que é mais estruturada
        else:
            self.meta_state.reasoning_trace.append("META: Alerta! Discordância detectada.")
            if "neural_symbolic_disagreement" not in self.meta_state.uncertainty_sources:
                self.meta_state.uncertainty_sources.append("neural_symbolic_disagreement")
            self.meta_state.confidence_in_decision = 0.3
            return f"Conflito de respostas. Neural: '{neural_solution}', Simbólica: '{symbolic_solution}'"


    def _is_problem_technical(self, text: str) -> float:
        technical_indicators = ["matemática", "equação", "lógica", "derivada", "integral", "algoritmo"]
        score = sum(1 for term in technical_indicators if term in text.lower())
        return min(1.0, score / 2.0)

    def _detect_ambiguity(self, text: str) -> float:
        ambiguous_words = ["pode", "talvez", "possivelmente", "geralmente", "acho que"]
        score = sum(1 for word in ambiguous_words if word in text.lower())
        return min(1.0, score / 2.0)

# ------------------------------------------------------------------------------
# 4. Exemplo de Uso (com um novo cenário para testar o fallback)
# ------------------------------------------------------------------------------
def run_simulation(problem_statement, context_info):
    print("="*60)
    print(f"PROBLEMA: '{problem_statement}'")
    print(f"CONTEXTO: {context_info}")
    print("-"*60)
    system = NeuroSymbolicMetaCognition()
    result = system.solve_problem(problem_statement, context_info)
    meta_state = result['meta_state']
    print("\n=== RESULTADO FINAL ===")
    print(f"Solução Proposta: {result['solution']}")
    print(f"Confiança Final: {meta_state.confidence_in_decision:.2f}")
    print(f"Estratégia(s) Tentada(s): {[s.value for s in meta_state.attempted_strategies]}")
    print("\n=== TRACE DE RACIOCÍNIO META-COGNITIVO (Transparência Real) ===")
    for i, step in enumerate(meta_state.reasoning_trace, 1):
        print(f"{i}. {step}")
    print("\n=== FONTES DE INCERTEZA IDENTIFICADAS ===")
    if meta_state.uncertainty_sources:
        for source in meta_state.uncertainty_sources:
            print(f"- {source}")
    else:
        print("Nenhuma fonte de incerteza foi registrada.")
    print("="*60 + "\n")

if __name__ == "__main__":
    # Cenário 1: Problema matemático claro (deve funcionar como antes)
    problem1 = "Em uma aula de matemática, a professora pergunta: Se x + 2 = 5, qual é o valor de x?"
    context1 = {"domain": "matemática", "source": "livro didático"}
    run_simulation(problem1, context1)

    # Cenário 2: Problema filosófico (deve funcionar como antes)
    problem2 = "Qual você acha que é o sentido da vida?"
    context2 = {"domain": "filosofia", "urgency": "baixa"}
    run_simulation(problem2, context2)

    # Cenário 3: O PROBLEMA QUE FALHOU ANTES (agora deve funcionar na primeira tentativa)
    problem3 = "Eu acho que em matemática, se x + 8 = 15, x talvez seja 7. Está certo?"
    context3 = {"domain": "verificação", "source": "aluno incerto"}
    run_simulation(problem3, context3)

    # Cenário 4: NOVO PROBLEMA PARA FORÇAR O FALLBACK
    # A regra simbólica não sabe o que é 'y'. Isso forçará um erro e a re-estratégia.
    problem4 = "Em lógica, se y + 10 = 22, o que é y?"
    context4 = {"domain": "lógica", "source": "quebra-cabeça"}
    run_simulation(problem4, context4)